In [2]:
import pandas as pd
import json, os

import tensorflow as tf
import numpy as np
import csv
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification #, BertModel, BertTokenizer, TFBertForSequenceClassification
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from collections import OrderedDict
import time
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, \
roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold
import random

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D

from sklearn.metrics import classification_report

C:\Users\iliaskaloup\Anaconda3\envs\tfgpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
seed = 123
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [4]:
model_variation = "neulab/codebert-cpp"
tokenizer = AutoTokenizer.from_pretrained(model_variation, do_lower_case=True) 

Downloading: 100%|████████████████████████████████████████████████████████████████| 1.50k/1.50k [00:00<00:00, 1.53MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 780k/780k [00:00<00:00, 1.87MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 446k/446k [00:00<00:00, 1.07MB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████| 2.01M/2.01M [00:00<00:00, 2.82MB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 280/280 [00:00<00:00, 278kB/s]


In [6]:
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [7]:
dataset = read_json_file("function.json")

In [29]:
df = pd.DataFrame(dataset)
#print(df)

In [30]:
df["target"].value_counts()

0    14858
1    12460
Name: target, dtype: int64

In [33]:
df['length'] = df['func'].apply(lambda x: len(str(x)))
max_length = df['length'].max()
#df['length'].idxmax()
print(df)
print(max_length)

      project                                 commit_id  target  \
0      FFmpeg  973b1a6b9070e2bf17d17568cbaf4043ce931f51       0   
1      FFmpeg  321b2a9ded0468670b7678b7c098886930ae16b2       0   
2      FFmpeg  5d5de3eba4c7890c2e8077f5b4ae569671d11cf8       0   
3      FFmpeg  32bf6550cb9cc9f487a6722fe2bfc272a93c1065       0   
4      FFmpeg  57d77b3963ce1023eaf5ada8cba58b9379405cc8       0   
...       ...                                       ...     ...   
27313    qemu  1ea879e5580f63414693655fcf0328559cdce138       0   
27314    qemu  f74990a5d019751c545e9800a3376b6336e77d38       0   
27315    qemu  a89f364ae8740dfc31b321eed9ee454e996dc3c1       0   
27316    qemu  39fb730aed8c5f7b0058845cb9feac0d4b177985       0   
27317  FFmpeg  7104c23bd1a1dcb8a7d9e2c8838c7ce55c30a331       0   

                                                    func  length  
0      static av_cold int vdadec_init(AVCodecContext ...    2221  
1      static int transcode(AVFormatContext **output_...   36

1095